In [1]:
import time
import requests
import pandas as pd

API_KEY = "XXXX"  # <-- put your key here (keep it local)
SYMBOLS = ["AAPL", "GOOGL", "NVDA"]
SLEEP_SECONDS = 15  # free tier safety

def fetch_income_statement(symbol: str) -> dict:
    url = "https://www.alphavantage.co/query"
    params = {"function": "INCOME_STATEMENT", "symbol": symbol, "apikey": API_KEY}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

all_rows = []

for symbol in SYMBOLS:
    print(f"Fetching income statement for {symbol}...")
    data = fetch_income_statement(symbol)

    if "quarterlyReports" not in data:
        print("  Unexpected response:", data)
        time.sleep(SLEEP_SECONDS)
        continue

    df = pd.DataFrame(data["quarterlyReports"])
    df["symbol"] = symbol
    all_rows.append(df)

    time.sleep(SLEEP_SECONDS)

is_df = pd.concat(all_rows, ignore_index=True)

# Keep only what we need
cols = ["symbol", "fiscalDateEnding", "totalRevenue", "grossProfit", "operatingIncome", "netIncome"]
is_df = is_df[[c for c in cols if c in is_df.columns]].copy()

# Convert types
is_df["fiscalDateEnding"] = pd.to_datetime(is_df["fiscalDateEnding"], errors="coerce")
for c in ["totalRevenue", "grossProfit", "operatingIncome", "netIncome"]:
    if c in is_df.columns:
        is_df[c] = pd.to_numeric(is_df[c], errors="coerce")

# Compute margins safely
is_df["grossMargin"] = is_df["grossProfit"] / is_df["totalRevenue"]
is_df["operatingMargin"] = is_df["operatingIncome"] / is_df["totalRevenue"]
is_df["netMargin"] = is_df["netIncome"] / is_df["totalRevenue"]

is_df = is_df.sort_values(["symbol", "fiscalDateEnding"], ascending=[True, False])

is_df.to_csv(
    r"C:\Users\adyat\OneDrive\Desktop\Adrian\LSE Data Analytics Career Accelerator\Employer Project\av_income_quarterly_margins.csv",
    index=False
)
print("Saved: av_income_quarterly_margins.csv")

is_df.head()

Fetching income statement for AAPL...
Fetching income statement for GOOGL...
Fetching income statement for NVDA...
Saved: av_income_quarterly_margins.csv


,symbol,fiscalDateEnding,totalRevenue,grossProfit,operatingIncome,netIncome,grossMargin,operatingMargin,netMargin
0,AAPL,2025-12-31,143756000000,69231000000,50852000000,42097000000,0.481587,0.353738,0.292836
1,AAPL,2025-09-30,102466000000,48341000000,32427000000,27466000000,0.471776,0.316466,0.268050
2,AAPL,2025-06-30,94036000000,43718000000,28202000000,23434000000,0.464907,0.299906,0.249202
3,AAPL,2025-03-31,95359000000,44867000000,29589000000,24780000000,0.470506,0.310291,0.259860
4,AAPL,2024-12-31,124300000000,58275000000,42832000000,36330000000,0.468825,0.344586,0.292277


In [3]:
is_df

,symbol,fiscalDateEnding,totalRevenue,grossProfit,operatingIncome,netIncome,grossMargin,operatingMargin,netMargin
0,AAPL,2025-12-31,143756000000,69231000000,50852000000,42097000000,0.481587,0.353738,0.292836
1,AAPL,2025-09-30,102466000000,48341000000,32427000000,27466000000,0.471776,0.316466,0.268050
2,AAPL,2025-06-30,94036000000,43718000000,28202000000,23434000000,0.464907,0.299906,0.249202
3,AAPL,2025-03-31,95359000000,44867000000,29589000000,24780000000,0.470506,0.310291,0.259860
4,AAPL,2024-12-31,124300000000,58275000000,42832000000,36330000000,0.468825,0.344586,0.292277
...,...,...,...,...,...,...,...,...,...
238,NVDA,2006-10-31,820572000,333942000,117613000,106511000,0.406962,0.143331,0.129801
239,NVDA,2006-07-31,687500000,292100000,95800000,86800000,0.424873,0.139345,0.126255
240,NVDA,2006-04-30,681807000,288757000,100685000,90676000,0.423517,0.147674,0.132994
241,NVDA,2006-01-31,633614000,254940000,112342000,98052000,0.402359,0.177304,0.154750
